Note: This notebook is an implementation of the problem collected in H. Paul Williams - Model Building in Mathematical Programming. 

# Description of the problem

In [61]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

Suppose a plumber stocks standard lengths of pipe, all of length 19 m. He has orders for

    1. 12 lengths of 4 m
    2. 15 lengths of 5 m
    3. 22 lengths of 6 m

# Solved by column generation

## The first round

Under column generation, we should turn to another way of formulation

$\gamma_{j}$: number of times pattern j is used
    
Then we should find minimum of such patterns
    
Minimize $\gamma_{1} + \gamma_{2} + \gamma_{3}$
subject to $2\gamma_{1} + 2\gamma_{2} + \gamma_{3} \ge 12$
$2\gamma_{1} + \gamma_{2} + 3\gamma_{3} \ge 15$
$\gamma_{2} \ge 22$

We can solve it with Pyomo and GLPK like below

In [62]:
# Initialization
model = ConcreteModel()
model.gamma_idx_set1 = Set(initialize=range(3)) 
model.gamma_var_set1 = Var(model.gamma_idx_set1, domain=NonNegativeIntegers)

# Set up objective
# var for var in model.gamma_var_set1 does not work as expected
model.obj = Objective(rule=sum(model.gamma_var_set1[i] for i in model.gamma_idx_set1), sense=minimize)

# Set up constraints
model.constraint_set1 = ConstraintList()
model.constraint_set1.add(2 * model.gamma_var_set1[0] + 2 * model.gamma_var_set1[1] + model.gamma_var_set1[2] >= 12)
model.constraint_set1.add(2 * model.gamma_var_set1[0] + model.gamma_var_set1[1] + 3 * model.gamma_var_set1[2] >= 15)
model.constraint_set1.add(model.gamma_var_set1[1] >= 22)

# Create solver and solve1
optm = SolverFactory('glpk')
result = optm.solve(model)

The optimal objective is 

In [63]:
result

{'Problem': [{'Name': 'unknown', 'Lower bound': 22.0, 'Upper bound': 22.0, 'Number of objectives': 1, 'Number of constraints': 3, 'Number of variables': 3, 'Number of nonzeros': 7, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.037573814392089844}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [64]:
value(model.obj)

22.0

The optimal solution for variables are

In [65]:
model.gamma_var_set1.extract_values()

{0: 0.0, 1: 22.0, 2: 0.0}

Therefore, we see that we arrive at 22 for pattern 2 as optima